In [1]:
import weaviate



In [2]:
auth_config = weaviate.AuthApiKey(api_key="")

In [4]:
import requests
import json

# download sample data 

In [6]:
# # Download the data
# resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
# data = json.loads(resp.text)  # Load data

In [5]:
data=[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressure',
  'Answer': 'wire'},
 {'Category': 'SCIENCE',
  'Question': 'In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance',
  'Answer': 'DNA'},
 {'Category': 'SCIENCE',
  'Question': 'Changes in the tropospheric layer of this are what gives us weather',
  'Answer': 'the atmosphere'},
 {'Category': 'SCIENCE',
  'Question': 'In 70-degree air, a plane traveling at about 1,130 feet per second breaks it',
  'Answer': 'Sound barrier'}]

In [7]:
#resp

<Response [200]>

In [6]:
data

[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressur

In [7]:
type(data)

list

In [8]:
# Parse the JSON and preview it
print(type(data), len(data))

<class 'list'> 10


In [9]:
def json_print(data):
    print(json.dumps(data, indent=2))

In [10]:

json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


# Create an embedded instance of Weaviate vector database

In [12]:
import weaviate, os
from weaviate import EmbeddedOptions
import openai

In [13]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
os.environ['OPENAI_API_KEY']=""
openai.api_key = os.environ['OPENAI_API_KEY']


# you can instantiate a Weaviate client to access your Weaviate cluster as follows.

In [14]:
# Instantiate the client
client = weaviate.Client(
    url="https://pepsicogenxvectorcb-ogajl9or.weaviate.network", # Replace w/ your Weaviate cluster URL
    auth_client_secret=auth_config,
    additional_headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY'] # Replace with your OpenAI key
        }
)

c:\Users\09029O744\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\ssl_.py:260: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = SSLContext(ssl_version or PROTOCOL_TLS)
c:\Users\09029O744\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py:407: DeprecationWarning: ssl.match_hostname() is deprecated
  match_hostname(cert, asserted_hostname)


In [15]:
client

# As you can see, we are using the OpenAI API key under additional_headers 
# to access the embedding model later. If you are using a different provider than OpenAI, 
# change the key parameter to one of the following that apply:
#     X-Cohere-Api-Key, X-HuggingFace-Api-Key, or X-Palm-Api-Key

In [16]:
print(f"Client created? {client.is_ready()}")

Client created? True


In [17]:
json_print(client.get_meta())

{
  "hostname": "http://[::]:8080",
  "modules": {
    "generative-cohere": {
      "documentationHref": "https://docs.cohere.com/reference/generate",
      "name": "Generative Search - Cohere"
    },
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "generative-palm": {
      "documentationHref": "https://cloud.google.com/vertex-ai/docs/generative-ai/chat/test-chat-prompts",
      "name": "Generative Search - Google PaLM"
    },
    "qna-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "OpenAI Question & Answering Module"
    },
    "ref2vec-centroid": {},
    "reranker-cohere": {
      "documentationHref": "https://txt.cohere.com/rerank/",
      "name": "Reranker - Cohere"
    },
    "text2vec-cohere": {
      "documentationHref": "https://docs.cohere.ai/embedding-wiki/",
      "name": "Cohere Module"


In [20]:
# ## check a random prompt answer
# # instruction for the generative module
# generatePrompt = "Describe the following as a Facebook Ad: {summary}"

In [22]:
# result = (
#   client.query
#   .get("Article", ["title", "summary"])
#   .with_generate(single_prompt=generatePrompt)
#   .with_near_text({
#     "concepts": ["Italian food"]
#   })
#   .with_limit(5)
# ).do()



# create question collection

In [18]:
# resetting the schema. CAUTION: This will delete your collection 
if client.schema.exists("Question"):
    client.schema.delete_class("Question")

In [19]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  # Use OpenAI as the vectorizer
    "moduleConfig": {
        "text2vec-openai": {
            "model": "ada",
            "modelVersion": "002",
            "type": "text"#,
            #"baseURL": os.environ["OPENAI_API_BASE"]
        }
    }
}

client.schema.create_class(class_obj)

In [23]:
# reminder for the data structure
json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [24]:
data

[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressur

In [25]:
with client.batch.configure(batch_size=5) as batch:
    for i, d in enumerate(data):  # Batch import data
        
        print(f"importing question: {i+1}")
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5


c:\Users\09029O744\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\ssl_.py:260: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = SSLContext(ssl_version or PROTOCOL_TLS)
c:\Users\09029O744\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py:407: DeprecationWarning: ssl.match_hostname() is deprecated
  match_hostname(cert, asserted_hostname)


importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [26]:
count = client.query.aggregate("Question").with_meta_count().do()
json_print(count)

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}


# Let's Extract the vector that represents each question

In [27]:
# write a query to extract the vector for a question
result = (client.query
          .get("Question", ["category", "question", "answer"])
          .with_additional("vector")
          .with_limit(1)
          .do())


In [29]:
result

{'data': {'Get': {'Question': [{'_additional': {'vector': [0.0076300455,
       0.00061342155,
       -0.02073168,
       0.0049861,
       0.01675592,
       0.024458133,
       -0.015115757,
       -0.03789436,
       -0.02800089,
       -0.02363149,
       0.0017861392,
       0.025665294,
       -0.010949738,
       -0.011684532,
       0.020154342,
       -0.014420327,
       0.0065245745,
       -0.0077153337,
       -0.010621705,
       0.014958301,
       -0.018815968,
       0.003939675,
       0.008266429,
       0.00072331255,
       -0.01007061,
       0.011461469,
       0.02582275,
       -0.015338819,
       -0.008935616,
       0.010083731,
       0.018501056,
       0.004369398,
       -0.041305903,
       0.017018346,
       -0.021479594,
       -0.03222595,
       -0.006639386,
       -0.015010785,
       0.006321194,
       -0.0022207827,
       0.019878794,
       0.009217724,
       -0.010556099,
       0.022109417,
       -0.020101856,
       0.0067246747,
      

In [28]:

json_print(result)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "vector": [
              0.0076300455,
              0.00061342155,
              -0.02073168,
              0.0049861,
              0.01675592,
              0.024458133,
              -0.015115757,
              -0.03789436,
              -0.02800089,
              -0.02363149,
              0.0017861392,
              0.025665294,
              -0.010949738,
              -0.011684532,
              0.020154342,
              -0.014420327,
              0.0065245745,
              -0.0077153337,
              -0.010621705,
              0.014958301,
              -0.018815968,
              0.003939675,
              0.008266429,
              0.00072331255,
              -0.01007061,
              0.011461469,
              0.02582275,
              -0.015338819,
              -0.008935616,
              0.010083731,
              0.018501056,
              0.004369398,
              -

# Query time

What is the distance between the query: biology and the returned objects?

In [30]:
response = (
    client.query
    .get("Question",["question","answer","category"])
    .with_near_text({"concepts": "biology"})
    .with_additional('distance')
    .with_limit(2)
    .do()
)



In [31]:
json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19695163
          },
          "answer": "DNA",
          "category": "SCIENCE",
          "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
        },
        {
          "_additional": {
            "distance": 0.20142835
          },
          "answer": "species",
          "category": "SCIENCE",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        }
      ]
    }
  }
}


In [32]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"]})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)



In [33]:
json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.18963969
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19144487
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20419747
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21438634
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
    

# We can let the vector database know to remove results after a threshold distance

In [34]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"], "distance": 0.24})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.18963969
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19144487
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20419747
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21438634
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
    

# Vector Databases support for CRUD operations

# create

In [41]:
#Create an object
object_uuid = client.data_object.create(
    data_object={
        'question':"Leonardo da Vinci was born in this country.",
        'answer': "Italy",
        'category': "Culture"
    },
    class_name="Question"
 )

In [42]:
print(object_uuid)

f1e8257d-1888-4744-bfae-8f7e85c269f5


# Read

In [48]:
data_object = client.data_object.get_by_id(object_uuid, class_name="Question")


In [49]:
json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1701861529990,
  "id": "f1e8257d-1888-4744-bfae-8f7e85c269f5",
  "lastUpdateTimeUnix": 1701861529990,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


In [45]:
#Create an object
object_uuid_1 = client.data_object.create(
    data_object={
        'question':"Asmita chatterjee is a data scientist.",
        'answer': "IBM",
        'category': "India"
    },
    class_name="Question_1"
 )

In [46]:
object_uuid_1

'ed8f2d87-0a5a-48c6-b19b-96545321df42'

In [52]:
data_object_1 = client.data_object.get_by_id(object_uuid_1, class_name="Question_1")

In [53]:
json_print(data_object_1)

{
  "class": "Question_1",
  "creationTimeUnix": 1701861544128,
  "id": "ed8f2d87-0a5a-48c6-b19b-96545321df42",
  "lastUpdateTimeUnix": 1701861544128,
  "properties": {
    "answer": "IBM",
    "category": "India",
    "question": "Asmita chatterjee is a data scientist."
  },
  "vectorWeights": null
}


### Update

In [54]:
client.data_object.update(
    uuid=object_uuid,
    class_name="Question",
    data_object={
        'answer':"Florence, Italy"
    })

In [55]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
)

json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1701861529990,
  "id": "f1e8257d-1888-4744-bfae-8f7e85c269f5",
  "lastUpdateTimeUnix": 1701861636394,
  "properties": {
    "answer": "Florence, Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


# Delete

In [57]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 12
          }
        }
      ]
    }
  }
}


In [58]:
client.data_object.delete(uuid=object_uuid, class_name="Question")

In [59]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 11
          }
        }
      ]
    }
  }
}


In [62]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"]})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)


In [63]:
response

{'data': {'Get': {'Question': [{'_additional': {'distance': 0.18963969},
     'answer': 'Elephant',
     'question': "It's the only living mammal in the order Proboseidea"},
    {'_additional': {'distance': 0.19144487},
     'answer': 'the nose or snout',
     'question': 'The gavial looks very much like a crocodile except for this bodily feature'},
    {'_additional': {'distance': 0.20419747},
     'answer': 'Antelope',
     'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa'},
    {'_additional': {'distance': 0.21438634},
     'answer': 'species',
     'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"},
    {'_additional': {'distance': 0.23649544},
     'answer': 'the diamondback rattler',
     'question': 'Heaviest of all poisonous snakes is this North American rattlesnake'},
    {'_additional': {'distance': 0.24689794},
     'answer': 'DNA',
     'question': 'In 